In [3]:
!pip install langchain chromadb faiss-cpu  tiktoken wikipedia

Defaulting to user installation because normal site-packages is not writeable
  Using cached wikipedia-1.4.0-py3-none-any.whl
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.2 MB 985.5 kB/s eta 0:00:18
   - -------------------------------------- 0.8/18.2 MB 1.0 MB/s eta 0:00:18
   -- ------------------------------------- 1.0/18.2 MB 1.1 MB/s eta 0:00:16
   -- ------------------------------------- 1.0/18.2 MB 1.1 MB/s eta 0:00:16
   -- ------------------------------------- 1.3/18.2 MB 1.0 MB/s eta 0:00:17
   --- ------------------------------------ 1.6/18.2 MB 1.0 MB/s eta 0:00:16
   ---- ----------------------------------- 1.8/18.2 MB 1.0 MB/s eta 0:00:16
   ---- ----------------------------------- 2.1/18.2 MB 1.1 MB/s eta 0:00:16
   ----- ---------------------------------- 2.

# Wikipedia Retriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [4]:
retriever = WikipediaRetriever(top_k_results=2, lang='en')

In [5]:
query = "who is the greatest captain indian cricket"

In [6]:
docs = retriever.invoke(query)

In [7]:
docs[0].page_content

"The India men's national cricket team, also known as Men in Blue, represents India in international cricket. It is governed by the Board of Control for Cricket in India (BCCI) and is a full member nation of the International Cricket Council with Test, ODI and T20I status. India are the current holders of the T20 World Cup, the Champions Trophy and the Asia Cup.\nThe team has played 594 Test matches, winning 183, losing 186, with 224 draws and 1 tie. As of August 2025, India is ranked fourth in the ICC Men's Test Team Rankings with 107 rating points. India have played in two of the three World Test Championship finals, finishing runners-up in 2021 and 2023, while finishing third in 2025.\nTest rivalries include the Border–Gavaskar Trophy with Australia, Freedom Trophy with South Africa, Anderson–Tendulkar Trophy with England.\nThe team has played 1,066 ODI matches, winning 567, losing 445, tying 10 and with 44 ending in a no-result. As of May 2025, India is ranked first in the ICC Men'

# Vector Store Retriever

In [16]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector data  base optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [85]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
vectorstroe = Chroma.from_documents(documents=documents, embedding=embedding_model, collection_name='mycollection')

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [26]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [28]:
results

[Document(metadata={}, page_content='Chroma is a vector data  base optimized for LLM-based search.'),
 Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

In [29]:
vectorstore.similarity_search(query=query)

[Document(metadata={}, page_content='Chroma is a vector data  base optimized for LLM-based search.'),
 Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.'),
 Document(metadata={}, page_content='OpenAI provides powerful embedding models.'),
 Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.')]

# MMR

In [31]:
from langchain.vectorstores import FAISS

In [30]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [32]:
vectorstore = FAISS.from_documents(
    documents=docs, 
    embedding=embedding_model
)

In [37]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
        search_kwargs={'k': 3}
)

In [38]:
query = "What is langchain?"
results = retriever.invoke(query)

In [39]:
results

[Document(id='461bb722-7ba2-4632-9fee-b5d38f88fef8', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='5fbd2c52-ad2e-45ac-a944-c0dafb4a51f0', metadata={}, page_content='MMR helps you get diverse results when doing similarity search.'),
 Document(id='9ef71aa1-56a2-4ce5-9492-78fcd1bf425a', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.')]

# Multiquery Retriever

In [51]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

True

In [41]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [42]:
vectorstore = FAISS.from_documents(
    embedding=embedding_model,
    documents=all_docs
)

In [46]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 6, 'lambda_mult': 0.25})

In [69]:
import os
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", api_key=api_key)

In [70]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=retriever
)

In [71]:
query = "How to improve energy levels and maintain balance?"

result = multiquery_retriever.invoke(query)

In [73]:
for ans in result:
    print(ans.page_content)

Drinking sufficient water throughout the day helps maintain metabolism and energy.
Consuming leafy greens and fruits helps detox the body and improve longevity.
Regular walking boosts heart health and can reduce symptoms of depression.
Mindfulness and controlled breathing lower cortisol and improve mental clarity.
Deep sleep is crucial for cellular repair and emotional regulation.
The solar energy system in modern homes helps balance electricity demand.


# ContextualCompressionRetriever

In [76]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [77]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [79]:
vectorstore = FAISS.from_documents(
    embedding=embedding_model,
    documents=docs
)

In [80]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [81]:
compressor = LLMChainExtractor.from_llm(llm)

In [82]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever
)

In [83]:
query = "What is photosynthesis?"
result = compression_retriever.invoke(query)

In [84]:
for ans in result:
    print(ans.page_content)

Photosynthesis is the process by which green plants convert sunlight into energy.
The chlorophyll in plant cells captures sunlight during photosynthesis.
